### Installing packages

In [ ]:
pip install faiss

     |████████████████████████████████| 4.7 MB 4.6 MB/s 


In [ ]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (219 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [ ]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 132 kB 5.3 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219329 sha256=ed1940d015e518e8793c4face19a8f8b0176b02fe40ec96d6681f83ea3663195
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


### Mount Google Drive and load embeddings
#### Merge all the different embeddings into one consolidated embedding 

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# DATA_PATH = "gdrive/MyDrive"

# with open(DATA_PATH +'/MultiBERT_Lasse.pkl','rb') as f:
#   embeddings = pickle.load(f)

In [ ]:
# DATA_PATH = "gdrive/MyDrive"

# with open(DATA_PATH +'/BERT_Lasse_Only.pkl','rb') as f:
#   embeddings = pickle.load(f)

In [ ]:
# DATA_PATH = "gdrive/MyDrive"

# with open(DATA_PATH +'/MED_BERT_Lasse_Only.pkl','rb') as f:
#   embeddings = pickle.load(f)

In [ ]:
DATA_PATH = "gdrive/MyDrive"

with open(DATA_PATH +'/lasse_embeddings.pkl','rb') as f:
  embeddings = pickle.load(f)

In [ ]:
print(len(embeddings["question"]))
print(len(embeddings["answer"]))
print(len(embeddings["question_embedding"]))
print(len(embeddings["answer_embedding"]))

29752
29752
29752
29752


In [ ]:
import numpy as np 
import faiss

In [ ]:
dictionary construct:
output = {
    "question": list(df["question"]),
    "answer": list(df["answer"]),
    "question_embedding": question_embeddings,
    "answer_embedding": answer_embeddings
}

In [ ]:
question_embedding = np.array(embeddings["question_embedding"])
answer_embedding = np.array(embeddings["answer_embedding"])
print(question_embedding.shape)
print(answer_embedding.shape)

(29752, 768)
(29752, 768)


In [ ]:
dimension = 768
nlist = 5          # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  
index_question = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)
index_answer = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

In [ ]:
print(index_question.is_trained)   # False
index_question.train(question_embedding)  # train on the database vectors
print(index_question.ntotal)   # 0
index_question.add(question_embedding)   # add the vectors and update the index
print(index_question.is_trained)  # True
print(index_question.ntotal)

False
0
True
29752


In [ ]:
print(index_answer.is_trained)   # False
index_answer.train(answer_embedding)  # train on the database vectors
print(index_answer.ntotal)   # 0
index_answer.add(answer_embedding)   # add the vectors and update the index
print(index_answer.is_trained)  # True
print(index_answer.ntotal)   # 29752

False
0
True
29752


## Similarity Search to generate new questions and answers pair

Search each questions with 3 nearest neighbours. Will only add to result array only if the normalised distances between the question and neighbour is more than 0.9.

In [ ]:
# nprobe = 2  # find 2 most similar clusters  
# k = 4  # return 3 nearest neighbours

# result = []

# for i in range(len(question_embedding)):
#   question_query_vector = question_embedding[i:i+1].astype('float32')
#   faiss.normalize_L2(question_query_vector)
#   question_distances, question_indices = index_question.search(question_query_vector, k)

#   for index in range(k):
#     if index == 0:
#       continue
#     q_index = question_indices[0][index]
#     if (q_index != i):          # possible to remove. TO TRY
#       if question_distances[0][index] > 0.9:
#         if embeddings["question"][i] != embeddings["question"][q_index] and embeddings["answer"][i] != embeddings["answer"][q_index]:
#           result.append((i, q_index))

Search each questions with 3 nearest neighbours with. Will only add to result array only if both neighbour's question and answer are in the results of question's question and answer similarity search.

In [ ]:
index_question.nprobe = 2  # find 2 most similar clusters
index_answer.nprobe = 2  
k = 4  # return 3 nearest neighbours

result = []

for i in range(len(question_embedding)):
  question_query_vector = question_embedding[i:i+1].astype('float32')
  question_distances, question_indices = index_question.search(question_query_vector, k)
  
  answer_query_vector = answer_embedding[i:i+1].astype('float32')
  answer_distances, answer_indices = index_answer.search(answer_query_vector, k)
  
  for index in range(k):
    q_index = question_indices[0][index]
    if (q_index != i) and q_index in answer_indices[0]:
      if embeddings["question"][i] != embeddings["question"][q_index] and embeddings["answer"][i] != embeddings["answer"][q_index]:
        result.append((i, q_index))

In [ ]:
print(result)

[(58, 4361), (158, 21229), (167, 21914), (186, 8999), (272, 152), (277, 710), (294, 747), (325, 7111), (359, 455), (369, 431), (374, 398), (381, 789), (395, 417), (401, 399), (431, 369), (437, 497), (455, 359), (497, 437), (597, 215), (655, 737), (737, 655), (747, 294), (789, 381), (790, 515), (790, 399), (834, 6460), (853, 960), (866, 1470), (900, 7161), (909, 7132), (909, 834), (924, 960), (924, 958), (958, 960), (965, 6986), (978, 7596), (988, 3274), (1009, 10185), (1039, 15067), (1060, 3176), (1063, 7807), (1063, 7540), (1066, 7806), (1066, 7520), (1079, 1196), (1084, 15255), (1084, 15254), (1090, 10138), (1196, 1079), (1202, 7982), (1211, 1230), (1215, 9663), (1230, 1258), (1238, 11023), (1240, 1232), (1241, 1204), (1243, 6910), (1250, 16086), (1254, 1796), (1258, 1230), (1270, 16084), (1311, 10466), (1330, 12738), (1345, 1384), (1347, 1333), (1370, 1390), (1383, 3959), (1384, 23271), (1390, 1370), (1402, 1341), (1413, 6507), (1427, 15944), (1438, 15523), (1438, 10416), (1470, 866

In [ ]:
print(len(result))

982


In [ ]:
print(embeddings["question"][186])
print(embeddings["answer"][186])

hello i'm a female 5'3 and going to be 19 next month is there anyway i can grow about 2 to 3 inches taller?
girls are done growing earlier than boys usually by age 18. that doesn't mean you won't see any growth but it's just unlikely. i would focus on accepting your body size and keeping it healthy. see webmd fit teen 's section for tips on healthy eating and physical activity.


In [ ]:
print(embeddings["question"][8999])
print(embeddings["answer"][8999])

hi im a 19 year old female who is 5 feet tall. is there anything i can take that will help me gain a few inches. please
unfortunately genetics generally determines how tall we will grow. at age 19 you're not likely to gain any more height. that said you might consult with your primary care provider to see if there are any options available to you that i'm not aware of (even the best nurse can't know about every new development in medicine! ). sometimes endocrine problems can cause people not to grow but these issues usually are identified and treated early on such as when puberty starts. i wish i had better news for you.


In [ ]:
for i in range(len(result)):
  first_index = result[i][0]
  second_index = result[i][1]

  first_q = embeddings["question"][first_index]
  first_a = embeddings["answer"][second_index]
  first_q_embeddings = embeddings["question_embedding"][first_index]
  first_a_embeddings = embeddings["answer_embedding"][second_index]

  second_q = embeddings["question"][second_index]
  second_a = embeddings["answer"][first_index]
  second_q_embeddings = embeddings["question_embedding"][second_index]
  second_a_embeddings = embeddings["answer_embedding"][first_index]

  embeddings["question"].append(first_q)
  embeddings["answer"].append(first_a)
  embeddings["question_embedding"].append(first_q_embeddings)
  embeddings["answer_embedding"].append(first_a_embeddings)

  embeddings["question"].append(second_q)
  embeddings["answer"].append(second_a)
  embeddings["question_embedding"].append(second_q_embeddings)
  embeddings["answer_embedding"].append(second_a_embeddings)

In [ ]:
print(len(embeddings["question"]))
print(len(embeddings["answer"]))
print(len(embeddings["question_embedding"]))
print(len(embeddings["answer_embedding"]))

31822
31822
31822
31822


### Save to Pickle

In [ ]:
pickle_out = open(DATA_PATH + "/FAISS_500_Med_BERT_Lasse_Only.pkl", "wb")
pickle.dump(embeddings, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()

### SAME QUESTIONS DIFFERENT ANSWER.

In [ ]:
print(embeddings["question"][26])
print(embeddings["answer"][26])

i eat 1 200-1 500 calories a day. is that enough? . i'm 19 5"3' female and weigh 120. for the past at least five years of my life i've been eating 1 200-1 500 calories which leaves me feeling satisfied energized and maintaining my weight. i've been eating a very healthy vegan diet and have eaten pretty healthy my whole life. i feel like i'm eating enough and i'm maintaining my weight but our "experts" say i need to eat more. i feel like if i ate more i would just gain weight but i'm at a healthy weight now. i exercise moderately.
calorie needs are individual and depend on age gender lean body mass and your level of activity. since you are not overweight i wonder why you count calories at all. it seems to me if you eat well balanced regular meals and let hunger and fullness guide your eating you will get the right amount for your body type. emerging research shows that young adults who eat this way (often called intuitive or mindful eating) have lower bmis -- and disordered eating -- th

In [ ]:
print(embeddings["question"][20854])
print(embeddings["answer"][20854])

i eat 1 200-1 500 calories a day. is that enough? . i'm 19 5"3' female and weigh 120. for the past at least five years of my life i've been eating 1 200-1 500 calories which leaves me feeling satisfied energized and maintaining my weight. i've been eating a very healthy vegan diet and have eaten pretty healthy my whole life. i feel like i'm eating enough and i'm maintaining my weight but our "experts" say i need to eat more. i feel like if i ate more i would just gain weight but i'm at a healthy weight now. i exercise moderately.
hope your weight is 120 pounds and not 120 kg. if the current calorie intake keeps you satisfied and energized you do not have to worry at all. if you are following any diet chart see to it that your calorie intake is between 1800-2100 calories which is normally recommended for a healthy person.


In [ ]:
print(embeddings["question"][20853])
print(embeddings["answer"][20853])

i eat 1 200-1 500 calories a day. is that enough? . i'm 19 5"3' female and weigh 120. for the past at least five years of my life i've been eating 1 200-1 500 calories which leaves me feeling satisfied energized and maintaining my weight. i've been eating a very healthy vegan diet and have eaten pretty healthy my whole life. i feel like i'm eating enough and i'm maintaining my weight but our "experts" say i need to eat more. i feel like if i ate more i would just gain weight but i'm at a healthy weight now. i exercise moderately.
calorie needs are individual and depend on age gender lean body mass and your level of activity. since you are not overweight i wonder why you count calories at all. it seems to me if you eat well balanced regular meals and let hunger and fullness guide your eating you will get the right amount for your body type. emerging research shows that young adults who eat this way (often called intuitive or mindful eating) have lower bmis -- and disordered eating -- th

In [ ]:
print(embeddings["question"][38])
print(embeddings["answer"][38])

what vitamins help migraines?
webmd has a list of vitamins and other supplements that might help with headaches. the page lists the supplements weight of the evidence and reviews. the link is below. vitamins and supplements for migraines. vitamin supplements are advised in migraines for its prevention. vitamins like vitamin b6 b12 vitamin c folic acid are proven useful to reduce the severity and frequency of migraine attacks. these supplements also prevent the migraine attacks and improves overall the quality of life.


In [ ]:
print(embeddings["question"][16217])
print(embeddings["answer"][16217])

what vitamins help migraines?
webmd has a list of vitamins and other supplements that might help with headaches. the page lists the supplements weight of the evidence and reviews. the link is below. vitamins and supplements for migraines.


### To test a query

In [ ]:
nprobe = 2  # find 2 most similar clusters  
k = 3  # return 3 nearest neighbours
question_query_vector = question_embedding[0:1].astype('float32')
question_distances, question_indices = index_question.search(question_query_vector, k)

In [ ]:
nprobe = 2  # find 2 most similar clusters  
k = 3  # return 3 nearest neighbours
answer_query_vector = answer_embedding[0:1].astype('float32')
answer_distances, answer_indices = index_answer.search(answer_query_vector, k)

In [ ]:
print(question_distances)
print(question_indices)
print()
print(answer_distances)
print(answer_indices)

[[ 0.       0.      22.07029]]
[[    0 29752  7002]]

[[ 0.       27.283146 28.226826]]
[[    0  4389 15665]]


In [ ]:
print(embeddings["question"][0])
print(embeddings["question"][7002])
print(embeddings["question"][19074])

my 5 1/2-year-old son displays adhd symptoms for 20 days then for 10 days he has none. is it adhd or another condition?
my 14-year-old son is taking an increased dose of concerta for adhd and has blurred vision in school. is this serious?
my 6 1/2-month-old has not been vaccinated for flu yet and his doctor says he should be. what is your advice?


In [ ]:
print(embeddings["answer"][0])
print(embeddings["answer"][4389])
print(embeddings["answer"][15665])

adhd and bipolar mood disorder (bmd) can coexist or be mistaken for one another. bmd usually is not diagnosed until young adulthood. however studies have shown that the earlier a person is diagnosed with bmd the more likely he is to have been diagnosed with adhd previously. in this case i would just like to reiterate that there is not enough information to discuss either possibility for your son. you mentioned that he becomes hyperactive for 3 weeks but not what his behaviors are like during those 10 days. you also do not mention irritability or mood swings just adhd symptoms. keep documenting the symptoms you are concerned about including what goes on in the home and at school when you see changes in behavior (do you work those weeks does he visit a relative or have a different aide in the classroom). you also mentioned that this began 7 months ago. i would also urge you to think about what also might have changed in your son's life about that time. consulting your pediatrician or a c